# US-Ignite Fed_Notebook

In [21]:
! pip install census

In [4]:
#import libraries from Carto.
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer
from cartoframes.data import Dataset
from cartoframes.viz.helpers import color_continuous_layer
import pandas as pd
import geopandas as gpd
import requests
import json
import censusdata
set_default_credentials(
    base_url='https://usignite-admin.carto.com/',
    api_key='81c922dc336bd82103d92a768816a4567c444909'
)
key='0995bcf2c5a4aa42899cf705840809959c68cf75'
import config
from census import Census
from us import states

In [26]:
config?

In [3]:
c = Census(MY_API_KEY)
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

NameError: name 'MY_API_KEY' is not defined

In [4]:
import pandas as pd

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [9]:
#make API calls with python


#allows us to store results of API call cleanly


#get list of all zipcodes in Los Angeles County separated by commas
laZips = open('laZips.txt', 'r').readlines()
laZips = [z.replace('\n', '') for z in laZips]
laZips = ','.join(laZips)

#put your census API key here
apiKey = "0995bcf2c5a4aa42899cf705840809959c68cf75"

#construct the API call we will use
baseAPI = "https://api.census.gov/data/2017/acs/acs5?key=%s&get=B01003_001E&for=zip%%20code%%20tabulation%%20area:%s" 
calledAPI = baseAPI % (apiKey, laZips)

#call the API and collect the response
response = requests.get(calledAPI)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#flip the order of the response from [population, zipcode] -> [zipcode, population]
formattedResponse = [item[::-1] for item in formattedResponse]

#store the response in a dataframe
laZipPopulations = pd.DataFrame(columns=['zipcode', 'population'], data=formattedResponse)

#save that dataframe to a CSV spreadsheet
laZipPopulations.to_csv('laZipPopulations.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'laZips.txt'

In [33]:
variables.keys()

dict_keys(['NAME', 'B01001_001E', 'B19013_001E'])

In [15]:
base = 'http://api.census.gov/data/'
years = ['2012']#['2009', '2012', '2015']
variables = {'NAME':'Name',
             'B01001_001E': 'Population total',
             'B19013_001E': 'Real Median Income',}
v = ','.join(variables.keys())
c = '*'
s = '*'

In [ ]:
	https://api.census.gov/data/2017/acs/acs1?get=NAME,B00001_001E&for=us:*&key=YOUR_KEY_GOES_HERE

In [19]:
df = pd.DataFrame()
for y in years:
    url = '{}{}/acs/acs5?get={}&for=county:{}&in=state:{}&key={}'.format(
        base, y, v, c, s, key)
    r = requests.get(url).json()
    dft = pd.DataFrame(r[1:], columns=r[0])
    dft['Year'] = y
    df = df.append(dft)
df = df.rename(columns=variables).set_index(
    ['Name', 'Year']).sort_index(level='Name')
df.head()

,,Population total,Real Median Income,state,county
Name,Year,,,,
"Abbeville County, South Carolina",2012,25387,35067,45,001
"Acadia Parish, Louisiana",2012,61611,38686,22,001
"Accomack County, Virginia",2012,33454,40780,51,001
"Ada County, Idaho",2012,394961,55499,16,001
"Adair County, Iowa",2012,7628,47872,19,001


IndentationError: unexpected indent (<ipython-input-17-12346824acf3>, line 2)

#load .env with credentials
from dotenv import load_dotenv
load_dotenv()
import os
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
USERNAME = os.getenv("USERNAME")

set_default_credentials(
    base_url=BASE_URL,
    api_key=API_KEY
)

credentials = Credentials(USERNAME, API_KEY)

## Extract: Bring GeoJson into Notebook

In [7]:
transit_stop_peak_arrival_gpdf = gpd.read_file('https://opendata.arcgis.com/datasets/0e7e79b428804da392abbb2f64cba7b2_253.geojson')

OSError: no such file or directory: 'opendata.arcgis.com/datasets/0e7e79b428804da392abbb2f64cba7b2_253.geojson'

## Transform
Add here some code to do data transformation (remove outliers, null values, etc.). It's probably best to do this type of work on the geopandas dataframe 

## Load: Upload to CARTO

In [ ]:
transit_stop_peak_arrival_d = Dataset(transit_stop_peak_arrival_gpdf)
transit_stop_peak_arrival_d.upload(table_name='portland_transit_stop_peak_arrival', if_exists='replace', credentials=credentials)

## Make a quick map using one of CARTO's helper functions

In [ ]:
transit_stop_peak_arrival_CARTO = Dataset('portland_transit_stop_peak_arrival')
Map(
    color_continuous_layer(transit_stop_peak_arrival_CARTO, 'pmpeakarrivals','PM Peak Arrivals')
)